In [1]:
from model import Model
from util import get_batch_dataset, get_dataset, get_record_parser, convert_tokens
import tensorflow as tf
from config import flags
import numpy as np
import json

In [2]:
flags.DEFINE_string('f', 'give up already', 'who cares lol')
config = flags.FLAGS

In [3]:
test_eval_file = 'data/og_check_test_meta.json'
test_record_file = 'data/og_check_test.tf'

with open(test_eval_file, "r") as fh:
    eval_file = json.load(fh)

meta = {'total': 1000}

In [4]:
with open(config.word_emb_file, "r") as fh:
    word_mat = np.array(json.load(fh), dtype=np.float32)
with open(config.char_emb_file, "r") as fh:
    char_mat = np.array(json.load(fh), dtype=np.float32)

In [5]:
test_batch = get_dataset('data/og_check_test.tf', get_record_parser(
        config, is_test=True), config).make_one_shot_iterator()

In [6]:
model = Model(config, test_batch, word_mat, char_mat, trainable=False)

Instructions for updating:
Use the retry module or similar alternatives.


In [7]:
sess_config = tf.ConfigProto(allow_soft_placement=True)
sess_config.gpu_options.allow_growth = True

sess = tf.Session(config=sess_config)

In [8]:
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

#### Restore trained bad pointer model!!
saver.restore(sess, tf.train.latest_checkpoint('log/retraining_model'))

#saver.restore(sess, tf.train.latest_checkpoint('log/model'))
sess.run(tf.assign(model.is_train, tf.constant(False, dtype=tf.bool)))

INFO:tensorflow:Restoring parameters from log/retraining_model/badptr-savepoint-5000


False

In [9]:
total = meta['total']
print(total)

1000


In [10]:
answer_dict = {}
remapped_dict = {}

for step in range(total // config.batch_size + 1):
    qa_id, loss, yp1, yp2 = sess.run([model.qa_id, model.loss, model.yp1, model.yp2])
    answer_dict_, remapped_dict_ = convert_tokens(eval_file, qa_id.tolist(), yp1.tolist(), yp2.tolist())
    answer_dict.update(answer_dict_)
    remapped_dict.update(remapped_dict_)

In [11]:
from util import metric_max_over_ground_truths, exact_match_score, f1_score

In [12]:
def evaluate(eval_file, answer_dict, only=None):
    f1 = exact_match = total = 0
    for key, value in answer_dict.items():
        if only == 'adv' and len(eval_file[key]['uuid'].split('-')) == 1:
            continue
        if only == 'orig' and len(eval_file[key]['uuid'].split('-')) > 1:
            continue
        total += 1
        ground_truths = eval_file[key]["answers"]
        prediction = value
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(f1_score,
                                            prediction, ground_truths)
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    return {'exact_match': exact_match, 'f1': f1}

In [13]:
metrics = evaluate(eval_file, answer_dict, only='orig')
print("Unmutated data")
print("Exact Match: {}, F1: {}".format(metrics['exact_match'], metrics['f1']))

Unmutated data
Exact Match: 64.11149825783973, F1: 71.41547590666262


In [14]:
metrics = evaluate(eval_file, answer_dict, only='adv')
print("Mutated data")
print("Exact Match: {}, F1: {}".format(metrics['exact_match'], metrics['f1']))

Mutated data
Exact Match: 52.78154681139756, F1: 61.187446079696095


In [15]:
metrics = evaluate(eval_file, answer_dict)
print("Overall")
print("Exact Match: {}, F1: {}".format(metrics['exact_match'], metrics['f1']))

Overall
Exact Match: 55.95703125, F1: 64.05409115815252
